In [1]:
!unzip /content/MovielensLatest_x1.zip

Archive:  /content/MovielensLatest_x1.zip
  inflating: valid.csv               
  inflating: test.csv                
  inflating: train.csv               


In [2]:
!git clone https://github.com/sn09/ranking.git

Cloning into 'ranking'...
remote: Enumerating objects: 299, done.
remote: Counting objects: 100% (299/299), done.
remote: Compressing objects: 100% (177/177), done.
remote: Total 299 (delta 136), reused 253 (delta 99), pack-reused 0 (from 0)
Receiving objects: 100% (299/299), 153.88 KiB | 9.05 MiB/s, done.
Resolving deltas: 100% (136/136), done.


In [3]:
import sys

import pandas as pd

sys.path.append("./ranking/models/")

from finalnet.model import FinalNet

In [4]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_val = pd.read_csv("valid.csv")

df_train.head()

,label,user_id,item_id,tag_id
0,0,84982,58,39525
1,0,87756,8757,22786
2,0,80311,14912,45676
3,1,82036,84509,33556
4,0,66867,57349,15092


In [5]:
for df in [df_train, df_val, df_test]:
    df["user_id"] = df["user_id"].astype("category")
    df["item_id"] = df["item_id"].astype("category")
    df["tag_id"] = df["tag_id"].astype("category")

In [10]:
import torch
from torch import nn


model = FinalNet(
    block_type="2B",
    use_field_gate=True,
    use_batch_norm=True,
    add_bias=True,
    block1_hidden_dims=[64, 64, 64],
    block1_hidden_activations=nn.ReLU,
    block1_dropout_rates=0.1,
    block2_hidden_dims=[64, 64, 64],
    block2_hidden_activations=nn.ReLU,
    block2_dropout_rates=0.1,
    residual_type="concat",
    proj_output_embeddings=False,
)

In [11]:
model

FinalNet(
  (loss_fn): BCEWithLogitsLoss()
)

In [12]:
import os

train_metrics, val_metrics = model.fit(
    features=df_train.drop(columns="label"),
    target=df_train["label"],
    val_features=df_val.drop(columns="label"),
    val_target=df_val["label"],
    optimizer_cls="torch.optim.Adam",
    optimizer_params=dict(lr=1e-3),
    num_epochs=8,
    seed=42,
    artifacts_path="./finalnet_artifacts",
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
    eval_metric_name="log_loss",
    eval_mode="min",
    embedded_features=["user_id", "item_id", "tag_id"],
    oov_masking_proba=0.05,
    default_embedding_size=10, # should be equal for all features if using field gate
)

[2025-05-03 00:08:15,101]{model.py:614} - INFO - Used features config: FeaturesConfig(features=[Feature(name='user_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=10, embedding_vocab_size=16976, embedding_padding_idx=None), Feature(name='item_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=10, embedding_vocab_size=23605, embedding_padding_idx=None), Feature(name='tag_id', feature_type=<FeatureType.CATEGORICAL: 'categorical'>, feature_size=1, needs_embed=True, embedding_size=10, embedding_vocab_size=49658, embedding_padding_idx=None)])
[2025-05-03 00:08:15,110]{model.py:629} - INFO - Artifacts path is /content/finalnet_artifacts
[2025-05-03 00:08:15,111]{model.py:635} - INFO - Best model path is finalnet_artifacts/best_model.pt
[2025-05-03 00:08:15,117]{model.py:640} - INFO - Building features mappings
[2025-05-03 00:08:15,118]{model.py:267} - INFO - Building mapping

Train epoch #0:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 00:10:12,780]{model.py:525} - INFO - Finished Train Epoch #0, average metrics - [loss: 0.62156]


Validation epoch #0:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 00:10:29,426]{model.py:525} - INFO - Finished Validation Epoch #0, average metrics - [AUC: 0.72580, log_loss: 0.55707]
[2025-05-03 00:10:39,846]{model.py:699} - INFO - Best model with log_loss = 0.5570680192117203 was saved to finalnet_artifacts/best_model.pt


...

[2025-05-03 00:25:56,228]{model.py:525} - INFO - Finished Validation Epoch #6, average metrics - [AUC: 0.92315, log_loss: 0.30358]
[2025-05-03 00:26:06,677]{model.py:699} - INFO - Best model with log_loss = 0.30358174897213464 was saved to finalnet_artifacts/best_model.pt


Train epoch #7:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 00:28:04,937]{model.py:525} - INFO - Finished Train Epoch #7, average metrics - [loss: 0.30926]


Validation epoch #7:   0%|          | 0/98 [00:00<?, ?it/s]

[2025-05-03 00:28:31,225]{model.py:525} - INFO - Finished Validation Epoch #7, average metrics - [AUC: 0.92610, log_loss: 0.30028]
[2025-05-03 00:28:41,755]{model.py:699} - INFO - Best model with log_loss = 0.30028135098411013 was saved to finalnet_artifacts/best_model.pt


In [13]:
test_metrics = model.test(
    features=df_test.drop(columns="label"),
    target=df_test["label"],
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
)

[2025-05-03 00:28:41,770]{model.py:735} - INFO - Building test dataloader
[2025-05-03 00:28:41,773]{model.py:292} - INFO - Encoding feature user_id
[2025-05-03 00:28:41,791]{model.py:292} - INFO - Encoding feature item_id
[2025-05-03 00:28:41,809]{model.py:292} - INFO - Encoding feature tag_id


Test epoch #-1:   0%|          | 0/49 [00:00<?, ?it/s]

[2025-05-03 00:28:50,784]{model.py:525} - INFO - Finished Test Epoch #-1, average metrics - [AUC: 0.92581, log_loss: 0.30061]


In [14]:
train_metrics_v2 = model.test(
    features=df_train.drop(columns="label"),
    target=df_train["label"],
    device="cuda:0",
    batch_size=4096,
    num_workers=2,
)

[2025-05-03 00:29:01,203]{model.py:735} - INFO - Building test dataloader
[2025-05-03 00:29:01,217]{model.py:292} - INFO - Encoding feature user_id
[2025-05-03 00:29:01,277]{model.py:292} - INFO - Encoding feature item_id
[2025-05-03 00:29:01,315]{model.py:292} - INFO - Encoding feature tag_id


Test epoch #-1:   0%|          | 0/343 [00:00<?, ?it/s]

[2025-05-03 00:29:55,528]{model.py:525} - INFO - Finished Test Epoch #-1, average metrics - [AUC: 0.94414, log_loss: 0.27193]


In [15]:
model

FinalNet(
  (loss_fn): BCEWithLogitsLoss()
  (embedding_layer): EmbeddingLayer(
    (output_proj): Identity()
    (embedding_modules): ModuleDict(
      (user_id): Embedding(16976, 10)
      (item_id): Embedding(23605, 10)
      (tag_id): Embedding(49658, 10)
    )
    (dummy_fn): Identity()
  )
  (fields_gate): FieldGate(
    (proj_field): Linear(in_features=3, out_features=3, bias=True)
  )
  (block1): FinalBlock(
    (net): Sequential(
      (0): FactorizedInteraction(
        (linear): Linear(in_features=60, out_features=64, bias=True)
        (activation): Identity()
      )
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Dropout(p=0.1, inplace=False)
      (4): FactorizedInteraction(
        (linear): Linear(in_features=64, out_features=64, bias=True)
        (activation): Identity()
      )
      (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (6): ReLU()
      (7):